## 补充
```
# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev) # 标准差
        tf.summary.scalar('max', tf.reduce_max(var)) # 最大值
        tf.summary.scalar('min', tf.reduce_min(var)) # 最小值
        tf.summary.histogram('histogram',var) # 直方图
```
* tf.summary.scalar(),用于收集标量信息
* tf.summary的简单用法：https://blog.csdn.net/Li_haiyu/article/details/89646820

### TensorFlow报错：InvalidArgumentError: You must feed a value for placeholder tensor 'inputs/y_input'
[https://blog.csdn.net/zsf442553199/article/details/79869377](https://blog.csdn.net/zsf442553199/article/details/79869377)<br />
<br />解决方法：
```
# tf==1.4.0
tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy', accuracy)
merged_summary = tf.summary.merge_all()
```
修改为
```
loss_summary = tf.summary.histogram("loss", loss)
accuracy_summary = tf.summary.histogram('accuracy', accuracy)
merged_summary = tf.summary.merge([accuracy_summary,loss_summary])
```



In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
# 载入数据集
mnist = input_data.read_data_sets("../MNIST_data", one_hot=True) 
# x相对路径  数据集下载到某一目录下（相对路径不懂可自行百度）
# one-hot=True 把标签设置为one-hot 格式

# 每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size  # 整除

# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev) # 标准差
        tf.summary.scalar('max', tf.reduce_max(var)) # 最大值
        tf.summary.scalar('min', tf.reduce_min(var)) # 最小值
        tf.summary.histogram('histogram',var) # 直方图


# 命名空间
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None,784],name="X-input")
    y = tf.placeholder(tf.float32, [None, 10],name='y-input')

# 创建一个简单的神经网络
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([1, 10]),name='b')
        variable_summaries(b)
    with tf.name_scope('xw_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)


# 代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(y-prediction))
    # tf.summary.scalar('loss', loss)
    loss_summary = tf.summary.histogram("loss", loss)
# 梯度下降法
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
# 初始化变量
init = tf.global_variables_initializer()


# 得到一个布尔型列表，存放结果是否正确
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1)) #argmax 返回一维张量中最大值索引
    # 求准确率
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) # 把布尔值转换为浮点型求平均数
        # tf.summary.scalar('accuracy', accuracy)
        accuracy_summary = tf.summary.histogram('accuracy', accuracy)

# 合并所有的summery
merged = tf.summary.merge([accuracy_summary,loss_summary])

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(51):
        for batch in range(n_batch):
            # 获得批次数据
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 运行网络并记录log
            summary,_ = sess.run([merged, train_step], feed_dict={x:batch_xs, y:batch_ys})
        # 记录变量
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + " Testing Accuracy: " + str(acc))

Extracting ../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data\t10k-labels-idx1-ubyte.gz
Iter 0 Testing Accuracy: 0.8322
Iter 1 Testing Accuracy: 0.8712
Iter 2 Testing Accuracy: 0.8817
Iter 3 Testing Accuracy: 0.8881
Iter 4 Testing Accuracy: 0.8938
Iter 5 Testing Accuracy: 0.8967
Iter 6 Testing Accuracy: 0.9009
Iter 7 Testing Accuracy: 0.9018
Iter 8 Testing Accuracy: 0.9036
Iter 9 Testing Accuracy: 0.9044
Iter 10 Testing Accuracy: 0.9064
Iter 11 Testing Accuracy: 0.9075
Iter 12 Testing Accuracy: 0.908
Iter 13 Testing Accuracy: 0.9089
Iter 14 Testing Accuracy: 0.9096
Iter 15 Testing Accuracy: 0.9102
Iter 16 Testing Accuracy: 0.9113
Iter 17 Testing Accuracy: 0.912
Iter 18 Testing Accuracy: 0.9133
Iter 19 Testing Accuracy: 0.9138
Iter 20 Testing Accuracy: 0.914
Iter 21 Testing Accuracy: 0.9144
Iter 22 Testing Accuracy: 0.9143
Iter 23 Testing Accuracy: 0.9153
Iter 24 Testing